In [1]:
%matplotlib inline
import os
os.chdir("/ddn1/vol1/staging/leuven/stg_00002/lcb/nhecker/software/git/pycisTopic")
from pycisTopic.cisTopicClass import *
from pycisTopic.qc import *
from pycisTopic.LDAModels import *
from pycisTopic.clustVis import *
from pycisTopic.topicFiltering import *
from pycisTopic.diffFeatures import *
from pycisTopic.pseudobulkPeakCalling import *
from pycisTopic.iterativePeakCalling import *
from pycisTopic.topicBinarization import *
from pycisTopic.SCopeLoomPy import *
import os
import pandas as pd
os.chdir("/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/example_files/Mult_GRN/from_loom/")
from export_to_loom import *

/staging/leuven/stg_00002/lcb/saibar/software/anaconda3/envs/pyCisTopic_env/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")
/staging/leuven/stg_00002/lcb/saibar/software/anaconda3/envs/pyCisTopic_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# file paths
path_rnaseq_loom = "/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/example_files/Mult_GRN/from_loom/ScoMAP_RNA_VE.loom"
path_atac_loom = "/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/example_files/Mult_GRN/from_loom/ScoMAP_ATAC_VE.loom"
path_tf_region_topic = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/example_files/Mult_GRN/from_files/TF-region.tsv"

# output file
path_outf = "/ddn1/vol1/staging/leuven/stg_00002/lcb/nhecker/scenic2/filtered_TF-region.tsv"

In [3]:
# parameters
min_rna_count = 1 # min. count to consider TF expressed in cell
min_cells = 10 # minimum number of cells with topic, reg. region and TF
min_fraction_tf = 0.75 # min fraction of cells with expressed TF per topic and reg. region

In [4]:
# get cell topic matrix
# Read loom
atac_loom = SCopeLoom.read_loom(path_atac_loom)
mtx_atac = atac_loom.ex_mtx.T
cell_topic_mat = mtx_atac[mtx_atac.index.str.contains('Topic')]

In [5]:
# From the scRNA-seq loom file, get expression matrix
# Read loom
rna_loom = SCopeLoom.read_loom(path_rnaseq_loom)
mtx_rna = rna_loom.ex_mtx.T

In [6]:
# read TF region topic file
df_tfreg = pd.read_csv(path_tf_region_topic, sep='\t')

### Binarize data and fraction of expressed TF per cell and topic 

In [7]:
# binarize topics per cell
arr_ct = np.asarray(cell_topic_mat)
l_norm = (arr_ct - np.min(arr_ct))/np.ptp(arr_ct)
thr = threshold_otsu(l_norm, nbins=100)

bin_cell_topic_mat = cell_topic_mat.copy()
bin_cell_topic_mat[ bin_cell_topic_mat >= thr] = 1
bin_cell_topic_mat[ bin_cell_topic_mat < thr] = 0

In [8]:
# binarize RNA expression
bin_mtx_rna = mtx_rna.copy()
bin_mtx_rna[bin_mtx_rna >= min_rna_count] = 1
bin_mtx_rna[bin_mtx_rna < min_rna_count] = 0

In [9]:
# compute overlap stats returns panda DataFrame
def get_topic_region_tf_fraction(tf_region_topic, binary_mtx_topic, binary_mtx_rna, topic_tag = "Topic"):
    
    # get shared cellIDs
    rna_cellids = binary_mtx_rna.columns
    topic_cellids = binary_mtx_topic.columns
    shared_cellids = list(set(rna_cellids).intersection(set(topic_cellids)))
    # subset topic matrix
    tbinary_mtx_topic = binary_mtx_topic.loc[:,shared_cellids]

    # initialize data frame
    nentry = tf_region_topic.shape[0]
    
    arr_n_cellsnp = np.empty(nentry) 
    np.copyto(arr_n_cellsnp, np.nan)
    arr_fraction_tf = np.empty(nentry) 
    np.copyto(arr_fraction_tf, np.nan)
        
    newdata = { 'fraction_tf' : arr_n_cellsnp, 'n_cells' : arr_n_cellsnp }
    resdf = pd.DataFrame(data=newdata, index=tf_region_topic.index)
    
    # loop over entries, get fraction of cells with TF
    for index in tf_region_topic.index:
        
        tf = tf_region_topic.loc[index, 'TF']
        reg = tf_region_topic.loc[index, 'region']
        topic = topic_tag + tf_region_topic.loc[index, 'topic']

        # extract cells with topic
        if topic in tbinary_mtx_topic.index and tf in binary_mtx_rna.index:
            
            cellids = tbinary_mtx_topic.loc[topic, (tbinary_mtx_topic.loc[topic,:] == 1) ].index
            arr_rna = np.asarray(binary_mtx_rna.loc[tf, cellids])
            
            n_cells = len(arr_rna)
            n_rna_cells = np.nan
            fraction = np.nan

            # count cells with TF for topic
            if n_cells > 0:
                n_rna_cells =  arr_rna.sum()
                fraction = n_rna_cells / n_cells
                
                # add entries to data frame
                resdf.loc[index, 'fraction_tf'] = fraction
                resdf.loc[index, 'n_cells'] = n_cells
            
    return resdf

In [10]:
# get fraction of TFs per Topic
df_fraction_tf = get_topic_region_tf_fraction(df_tfreg, bin_cell_topic_mat, bin_mtx_rna)

In [11]:
df_tfreg = pd.concat([df_tfreg, df_fraction_tf], axis=1)

In [12]:
# filter data frame
df_fil_tfreg = df_tfreg.loc[ ( (df_tfreg.n_cells >= min_cells) & (df_tfreg.fraction_tf >= min_fraction_tf) ), :]
df_fil_tfreg

,TF,region,topic,fraction_tf,n_cells
6,hth,chr2L:13084310-13086017,14,1.000000,62.0
10,exd,chr2L:13084310-13086017,14,0.935484,62.0
251,jim,chr2L:7423403-7426242,27,0.944444,36.0
315,RpII215,chr2L:7423403-7426242,27,0.777778,36.0
334,gl,chr2L:7423403-7426242,12,0.982699,578.0
...,...,...,...,...,...
573560,maf-S,chrX:9615301-9616575,48,0.800000,35.0
573562,cnc,chrX:9615301-9616575,48,0.885714,35.0
573565,Jra,chrX:9615301-9616575,48,0.914286,35.0
573570,pan,chrX:9615301-9616575,48,0.828571,35.0


In [13]:
# write output
df_fil_tfreg.to_csv(path_outf, sep="\t")
print("Output written to: " + path_outf)

Output written to: /ddn1/vol1/staging/leuven/stg_00002/lcb/nhecker/scenic2/filtered_TF-region.tsv
